# Web Scraping

## Importation des librairies

In [20]:
# a mettre au propre une fois le projet fini, pour l'instant vide
# vcar selon le besoin au moment venue:
# voir si natif d'Anaconda sinon l'importer par la cmd

## Creation d'une methode generale

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
    
Evitez d'utiliser l'import de tous les éléments d'un module (\*)
    
Préférez le fait de donner un alias court à votre module de manière à bien voir dans le code quel élément vient d'où :
    
```python
import peewee as pw

class Annonce(pw.Model):
    id = pw.CharField(unique=True, primary_key=True)
...
```
</div>

In [1]:
from playhouse.sqlite_ext import SqliteExtDatabase

# à importer par la cmd
from peewee import *

In [2]:
# creation de la base sql
db = SqliteExtDatabase('database.sqlite')

In [3]:
# creation de classe: nouveau type d'objet
class Annonce(Model):
    # id = "pap-123456789"
    id = CharField(unique=True, primary_key=True)
    # site = [pap, lbc, logic-immo, seloger]
    site = CharField()
    created = DateTimeField()
    title = CharField()
    description = TextField(null=True)
    telephone = TextField(null=True)
    price = FloatField()
    charges = FloatField(null=True)
    surface = FloatField()
    rooms = IntegerField()
    bedrooms = IntegerField(null=True)
    city = CharField()
    link = CharField()
    picture = CharField(null=True)
    posted2trello = BooleanField(default=False)

    class Meta:
        database = db
        order_by = ('-created',)


def create_tables():
    with db:
        db.create_tables([Annonce])

## Scrapping des différents sites: Seloger.com, LeBonCoin.fr, Biend'Ici

### Scraping SeLoger.com

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">       
Je vous recommande vraiment d'utiliser BeautifulSoup (bs4) qui est plus simple à utiliser que xml.etree.ElementTree et ne bug pas au moindre défaut de code HTML...
    
https://www.crummy.com/software/BeautifulSoup/bs4/doc
</div>

In [4]:
import requests

import xml.etree.ElementTree as ET

from datetime import datetime

In [5]:
#module qui récupère les annonces de SeLoger.com

def search(parameters):
    # Préparation des paramètres de la requête
    payload = {
        'px_loyermin': parameters['price'][0],
        'px_loyermax': parameters['price'][1],
        'surfacemin': parameters['surface'][0],
        'surfacemax': parameters['surface'][1],
        # Si parameters['rooms'] = (2, 4) => "2,3,4"
        'nbpieces': list(range(parameters['rooms'][0], parameters['rooms'][1] + 1)),
        # Si parameters['bedrooms'] = (2, 4) => "2,3,4"
        'nb_chambres': list(range(parameters['bedrooms'][0], parameters['bedrooms'][1] + 1)),
        'ci': [int(cp[2]) for cp in parameters['cities']]
    }
    
     # Insertion des paramètres propres à LeBonCoin
    payload.update(parameters['seloger'])

    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.167 Safari/537.36'}

    request = requests.get("https://www.seloger.com/list.htm?types=1&projects=2,5&enterprise=0&natures=1,2,4&places=[{ci:750056}]&qsVersion=1.0", params=payload, headers=headers)
    
    xml_root = ET.fromstring(request.text)
    
    for annonceNode in xml_root.findall('annonces/annonce'):
        # Seconde requête pour obtenir la description de l'annonce
        _payload = {'noAudiotel': 1, 'idAnnonce': annonceNode.findtext('idAnnonce')}
        _request = requests.get("http://ws.seloger.com/annonceDetail_4.0.xml", params=_payload, headers=headers)
        
        photos = list()
        for photo in annonceNode.find("photos"):
            photos.append(photo.findtext("stdUrl"))

        annonce, created = Annonce.create_or_get(
            id='seloger-' + annonceNode.find('idAnnonce').text,
            site='SeLoger',
            # SeLoger peut ne pas fournir de titre pour une annonce T_T
            title="Appartement " + annonceNode.findtext('nbPiece') + " pièces" if annonceNode.findtext('titre') is None else annonceNode.findtext('titre'),
            description=ET.fromstring(_request.text).findtext("descriptif"),
            telephone=ET.fromstring(_request.text).findtext("contact/telephone"),
            created=datetime.strptime(annonceNode.findtext('dtCreation'), '%Y-%m-%dT%H:%M:%S'),
            price=annonceNode.find('prix').text,
            charges=annonceNode.find('charges').text,
            surface=annonceNode.find('surface').text,
            rooms=annonceNode.find('nbPiece').text,
            bedrooms=annonceNode.find('nbChambre').text,
            city=annonceNode.findtext('ville'),
            link=annonceNode.findtext('permaLien'),
            picture=photos
        )

        if created:
            annonce.save() 

### Scraping LeBonCoin.fr

In [6]:
# Virginie

### Scraping BienDici.com

In [7]:
# Virginie et moi à faire

## Programme principal: pour l'instant fait que sur seloger

In [8]:
import os
import sys
import json

In [9]:
import logging

In [10]:
logging.basicConfig(level=logging.INFO)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
A priori, la case ci-dessous n'a pas lieu d'être...</div>

In [13]:
# os.chdir(os.path.dirname(sys.argv[0]))

In [14]:
create_tables()

In [15]:
print(db)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Evitez d'utiliser des chemins d'accès absolus qui bien sûr ne marcheront que si vous êtes sur votre machine... 
Et stockez vos fichiers et résultats dans le même dossier, ou encore mieux, dans un sous-dossier du dossier contenant votre Notebook</div>

In [16]:
# Chargement des paramètres de recherche depuis le fichier JSON
# with open("C:/Users/asial/projet_ws/parameters.json", encoding='utf-8') as parameters_data:
with open("parameters.json", encoding='utf-8') as parameters_data:
    parameters = json.load(parameters_data)

In [17]:
# Recherche et insertion en base
if "seloger" in parameters['ad-providers']:
    logging.info("Retrieving from seloger")
    search(parameters)

INFO:root:Retrieving from seloger


ParseError: not well-formed (invalid token): line 605, column 62 (<string>)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
On va analyser ce qui se passe en détail
Créons une "zone de travaux" dans notre code, bien délimitée pour bien mettre en avant que le code ci-dessous est temporaire</div>

# Work in Progress

In [51]:
#module qui récupère les annonces de SeLoger.com

# def search(parameters):
# Préparation des paramètres de la requête
payload = {
    'px_loyermin': parameters['price'][0],
    'px_loyermax': parameters['price'][1],
    'surfacemin': parameters['surface'][0],
    'surfacemax': parameters['surface'][1],
    # Si parameters['rooms'] = (2, 4) => "2,3,4"
    'nbpieces': list(range(parameters['rooms'][0], parameters['rooms'][1] + 1)),
    # Si parameters['bedrooms'] = (2, 4) => "2,3,4"
    'nb_chambres': list(range(parameters['bedrooms'][0], parameters['bedrooms'][1] + 1)),
    'ci': [int(cp[2]) for cp in parameters['cities']]
}

 # Insertion des paramètres propres à LeBonCoin
payload.update(parameters['seloger'])

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.167 Safari/537.36'}

request = requests.get("https://www.seloger.com/list.htm?types=1&projects=2,5&enterprise=0&natures=1,2,4&places=[{ci:750056}]&qsVersion=1.0", params=payload, headers=headers)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Déjà, comme expliqué plus haut, utilisez BeautifulSoup!</div>

In [54]:
from bs4 import BeautifulSoup
# xml_root = ET.fromstring(request.text)
xml_root = BeautifulSoup(request.text)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
On écrit le résultat pretty-fié dans un fichier pour le visualiser sous Notepad++ :</div>

In [61]:
with open('temporary.html', 'w', encoding='utf-8') as dest:
    dest.write(xml_root.prettify())

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
    
Et on observe sous Notepad++: 
- qu'il y a un json dans votre HTML qui contient plein de données (ligne 1063 - 3243) !
- que ce json est enfermé dans du code javascript, et qu'il est précédé d'une chaine de caractère :

```javascript
var ava_data =```

- et qu'il est suivi par la chaine de caractère :

```javascript
ava_data.logged =```

</div>

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Essayons de ne garder que ce json : </div>

In [96]:
clean_json = request.text.split("var ava_data = ")[1].split("ava_data.logged")[0]

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Ecrivons le résultat dans un fichier pour visualiser le contenu sous Notepad++</div>

In [95]:
with open('temporary.json', 'w', encoding='utf-8') as dest:
    dest.write(clean_json)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Trop de caractères de retour à la ligne qui polluent le contenu. 
    
Améliorons notre extraction :</div>

In [103]:
clean_json = request.text.split("var ava_data = ")[1].split("ava_data.logged")[0].strip()

In [104]:
with open('temporary.json', 'w', encoding='utf-8') as dest:
    dest.write(clean_json)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Essayons maintenant de charger cette chaine de caractères avec le module json :</div>

In [106]:
json_obj = json.loads(clean_json)

JSONDecodeError: Extra data: line 1091 column 2 (char 38199)

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Problème avec le caractère 38199. Bizarre...

Il y a combien de caractère dans clean_json ?</div>

In [107]:
len(clean_json)

38200

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Donc c'est juste le dernier caractère qui pose problème! C'est quoi ce caractère? </div>

In [109]:
clean_json[-1]

';'

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
En javascript, on met des points-virgule en fin de ligne... Il suffit de l'enlever</div>

In [110]:
json_obj = json.loads(clean_json[:-1])

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
Et là, magnifique, le rêve de tout scraper! Un json de données ultra-propres et extrêmement faciles à exploiter!</div>

In [111]:
json_obj

{'search': {'levier': 'Recherche depuis la liste',
  'nbresults': '11\xa0442',
  'nbpage': '1',
  'typedetransaction': ['vente', 'viager'],
  'nbpieces': [''],
  'typedebien': ['Appartement'],
  'pays': 'FR',
  'nbchambres': [''],
  'budget': {'min': '', 'max': ''},
  'surface': {'min': '', 'max': ''},
  'surface_terrain': {'min': '', 'max': ''},
  'type': [{'name': 'Nouvelle recherche', 'value': True},
   {'name': 'Dernière recherche', 'value': False},
   {'name': 'Recherche enregistrée', 'value': False}],
  'etage': {'min': '', 'max': ''},
  'tri': {'criteria': 'Sélection', 'direction': ''},
  'chauffage': [{'name': 'individuel', 'value': False},
   {'name': 'central', 'value': False},
   {'name': 'electrique', 'value': False},
   {'name': 'gaz', 'value': False},
   {'name': 'fuel', 'value': False},
   {'name': 'sol', 'value': False},
   {'name': 'radiateur', 'value': False}],
  'cuisine': [{'name': 'cuisine separee', 'value': False},
   {'name': 'cuisine americaine', 'value': False}

<div style="background-color: #dff0d8;border: 1px solid transparent;border-color: #d6e9c6;border-radius: 10px;color: #3c763d;font-size: 14px;line-height: 20px;margin: 7px 3px;padding: 15px;">
A vous de jouer maintenant...
   
Bon courage pour la suite !
</div>

# End work

In [2]:
test=parameters.get('cities')

In [3]:
print(test)

['Nanterre', 92000, 920050]


In [4]:
type(test[1])

int

In [5]:
for cp in parameters['cities']:
    print(cp)

Nanterre
92000
920050


In [6]:
print(test[1])

92000


Pourquoi dans la fonction search 'ci' récupère les éléments de Nanterre et non de la liste [] donc 92000 ??? T_T